In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np
import mysql.connector
import io


data_folder    = "../data/mysql_extractions/"
info_folder    = "../data/info/"
queries_folder = "../queries/"

pd.options.display.max_columns = None
pd.options.display.max_rows    = None

def export(table_name, cnx, filename = None, abbr = None, query= None):
    
    if not query:
        query = f"SELECT * FROM {table_name}"
        
    results = pd.read_sql_query(query, cnx)
    
    suffix = ""
    if abbr:
        suffix = "_"+abbr
        
    if not filename:
        filename = table_name
    
    path = f"{data_folder}{filename}{suffix}.csv"
    
    results.to_csv(path, index=False)
    print(str(len(results)) + ' rows written successfully to ' + path)

In [2]:
from configparser import ConfigParser

#Read data_info.ini file
inifile="../report/data_info.ini"
config_object = ConfigParser()
config_object.read(inifile)

#Get the DATA_INFO section
datainfo = config_object["DATA_INFO"]

# DB Structure

# DB Extraction

## Connection

In [2]:
DBNAME = "dualt_global"
DBABBR = ""

config = {
  'user': 'root',
  'password': '',
  'host': 'localhost',
  'database': DBNAME,
  'raise_on_warnings': True
}

try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)

## Activities Table

### Extraction of single activities and users data

You can check the queries here: [activities and users](/queries/elements_details.html) and [months](/queries/month_details.html) (or download the .sql files: [activities and users](/queries/elements_details.sql) and [months](/queries/month_details.sql)).

In [3]:
with open(f'{queries_folder}elements_details.sql', 'r') as file:
    query = file.read()

In [4]:
results = cnx.cmd_query_iter(query)
for i,cur in enumerate(results,1):
    print(f'cursor {i}', end=" OK. ")
cnx.commit()

cursor 1 OK. cursor 2 OK. 

In [5]:
export("V_activities_info",cnx,"activities",DBABBR)
export("V_user_info",cnx,"users",DBABBR)

45754 rows written successfully to ../data/mysql_extractions/activities.csv
1297 rows written successfully to ../data/mysql_extractions/users.csv


We ended up two files **activities.csv** and **users.csv** with the following structure:

In [26]:
users_info = pd.read_csv(f"{info_folder}users_info.csv",sep=";").set_index('attribute')
users_info

,type,description
attribute,,
us_user,numeric,user id
user_email,nominal,user email
user_name,nominal,first and last name
archived,ordinal,"boolean: if a user is archived, is no more vis..."
user_type,mixed,types of the user in csv
classes,mixed,classes to which the user belongs
companies,mixed,companies to which the user belongs
n_activities,interval,the number of user activities
n_activities_school_year_1,interval,the number of user activities on 1st year


In [24]:
users_info = pd.read_csv(f"{info_folder}activities_info.csv",sep=";").set_index('attribute')
users_info

,type,description
attribute,,
ac_activity,numeric,activity id
us_user,numeric,user id
ac_title,nominal,activity's title
at_activityType,categorical,"type of the activity ['recipe','experience']"
len_description,interval,the length (in words) of the description
ac_atSchool,ordinal,"boolean, if the recipe was done at school"
ac_atInteraziendale,ordinal,"boolean, if the recipe was done in an ""special..."
ac_atCompany,ordinal,"boolean, the recipe was done in the company"
avg_step_len,interval,average step length (in words)


### Extraction of months informations

In [ ]:
with open(f'{queries_folder}month_details.sql', 'r') as file:
    query = file.read()

results = cnx.cmd_query_iter(query)
for i,cur in enumerate(results,1):
    print(f'cursor {i}', end=" OK. ")
cnx.commit()

In [7]:
export("V_months_info",cnx,filename="months",abbr=DBABBR)

36 rows written successfully to ../data/mysql_extractions/months.csv


In [33]:
months_info = pd.read_csv(f"{info_folder}months_info.csv",sep=";").set_index('attribute')
months_info

,type,description
attribute,,
month,categorical,month from 1 to 12
activity_school_year,interval,vintage
start_year,useless,residual from queries
n_users_per_year,interval,how many users in total are passed from the sc...
n_logins,interval,total number of logins
n_activities,interval,total number of activities (recipes + experien...
n_recipes,interval,total number of recipes
n_experiences,interval,total number of experiences
avg_n_user_activities,interval,average number of activities per user


### Extraction of feedbacks details

In [29]:
cursor = cnx.cursor()

query = ("SELECT no_idObject as ac_activity, no_sender, no_recipient, "
             "no_type, no_timestamp, no_isRead, au_evaluation, tr.ar_evaluation "
         "FROM notifications AS n "
             "RIGHT JOIN activities_users ON n.no_idObject = activities_users.ac_activity "
             "RIGHT JOIN users ON activities_users.us_user = users.us_user "
             "RIGHT JOIN activities_responsables AS tr ON tr.ac_activity = n.no_idObject "
         "WHERE no_type = 'feedback response' OR no_type='feedback request' "
         "ORDER BY no_timestamp ASC "
        )

cursor.execute(query)

activities = {}
for (ac_activity, no_sender, no_recipient, no_type, no_timestamp, no_isRead, au_evaluation, ar_evaluation) in cursor:
    ac_activity = int(ac_activity)
    if not activities.get(ac_activity):
        activities[ac_activity] = {
            'requests':[],
            'responses':[], 
            'edits': [],
            'sender':None,
            'recipient':None,
            'supervisor_grade':None,
            'student_grade':None
        }
    
    action = 'responses'
    recipient = no_sender
    sender = no_recipient
    if no_type == 'feedback request':
        action = 'requests'
        recipient = no_recipient
        sender = no_sender
        
    activities[ac_activity]['sender'] = sender
    activities[ac_activity]['recipient'] = recipient
    activities[ac_activity]['student_grade'] = au_evaluation
    activities[ac_activity]['supervisor_grade'] = ar_evaluation
        
    activities[ac_activity][action].append(no_timestamp)

cursor.close()

True

In [30]:
cursor = cnx.cursor()

query = ("""
        SELECT ac_activity, data 
        FROM LOG_activities 
         WHERE operazione='U'
         """)

cursor.execute(query)

for (ac_activity, data) in cursor:
    if not activities.get(ac_activity):
        continue
        
    activities[ac_activity]['edits'].append(data)

cursor.close()

True

In [31]:
activities_feedbacks_info = []
for activity,v in activities.items():
    requests    = v['requests']
    responses   = v['responses']
    edits       = v['edits']
    sender      = v['sender']
    recipient   = v['recipient']
    student_grade    = v['student_grade']
    supervisor_grade = v['supervisor_grade']
    end_date  = None
    no_end    = False
    
    for request_date in requests:
        start_date = request_date
        if no_end or (end_date and request_date < end_date):
            continue
        end_date = None
        no_end   = True
        for response_date in responses:
            if response_date < request_date:
                continue
            no_end   = False
            end_date = response_date
            break
        
        request_times = 1
        for _request_date in requests:
            if _request_date > start_date and (no_end or _request_date < end_date):
                request_times+=1
                
        delay_hours = None
        if end_date:
            delay_hours = round((end_date-start_date).total_seconds()/60/60)
            
        edits_between = 0
        edits_after   = 0
        if end_date:
            next_response_date = None
            
            for response_date in responses:
                if response_date <= end_date:
                    continue
                next_response_date = response_date
                break
            
            
            for edit_date in edits:
                if edit_date > start_date and edit_date < end_date:
                    edits_between += 1
                if edit_date > end_date and (not next_response_date or edit_date < next_response_date):
                    edits_after += 1
                
            
        activity_info = {
                         'ac_activity':activity, 'sender': sender, 'recipient': recipient,
                         'request_date':start_date,'response_date':end_date, 'delay_hours':delay_hours,
                         'times_before_answer':request_times,
                         'edits_between': edits_between, 'edits_after': edits_after,
                         'student_grade': student_grade, 'supervisor_grade':supervisor_grade
                        }
        
        activities_feedbacks_info.append(activity_info)

In [32]:
cursor = cnx.cursor()

try:
    cursor.execute("DROP TABLE IF EXISTS activities_feedbacks_info; ");
except Exception as e:
    print(e)

query = (
         "CREATE TABLE activities_feedbacks_info "
         "     ( ac_activity INT(11) NOT NULL , "
         "       sender INT(11) NOT NULL , "
         "       recipient INT(11) NOT NULL , "
         "       request_date TIMESTAMP NOT NULL , "
         "       response_date TIMESTAMP NULL DEFAULT NULL , "
         "       delay_hours INT(5) NULL DEFAULT NULL , "
         "       edits_between INT(3) NULL DEFAULT NULL , "
         "       edits_after INT(3) NULL DEFAULT NULL , "
         "       times_before_answer INT(3) NOT NULL DEFAULT '1', "
         "       student_grade DECIMAL(3,2) NULL DEFAULT NULL , " 
         "       supervisor_grade DECIMAL(3,2) NULL DEFAULT NULL , " 
         "INDEX (ac_activity)) ENGINE = InnoDB;"
        )

cursor.execute(query);

In [33]:
cursor.executemany("""
    INSERT INTO activities_feedbacks_info 
        (ac_activity, sender, recipient, 
            request_date, response_date, 
            delay_hours, edits_between, 
            edits_after, times_before_answer,
            student_grade, supervisor_grade
        )
    VALUES (
        %(ac_activity)s, %(sender)s, %(recipient)s, 
        %(request_date)s, %(response_date)s, %(delay_hours)s, 
        %(edits_between)s, %(edits_after)s, %(times_before_answer)s,
        %(student_grade)s, %(supervisor_grade)s)""", activities_feedbacks_info)

cursor.close()

True

In [34]:
cnx.commit() 

In [35]:
query = ("SELECT info.*, a.activity_school_year, a.start_year "
         "FROM activities_feedbacks_info AS info "
         "NATURAL JOIN activities_users "
         "NATURAL LEFT JOIN V_all_activities_users AS a "
         "ORDER BY info.request_date"
        )
export("",cnx,filename="activities_feedbacks_info",abbr=DBABBR,query=query)

5198 rows written successfully to ../data/mysql_extractions/activities_feedbacks_info.csv


In [36]:
df = pd.read_csv(f"{data_folder}activities_feedbacks_info.csv")
df.head()

,ac_activity,sender,recipient,request_date,response_date,delay_hours,edits_between,edits_after,times_before_answer,student_grade,supervisor_grade,activity_school_year,start_year
0,438,20,86,2014-02-04 13:57:46,NaN,NaN,0,0,1,4.0,NaN,1,2013
1,579,20,86,2014-02-04 15:20:18,NaN,NaN,0,0,1,4.5,NaN,1,2013
2,723,65,87,2014-02-20 14:42:12,2014-11-17 20:59:09,6486.0,1,0,1,5.0,4.0,1,2013
3,844,20,86,2014-02-25 13:32:38,NaN,NaN,0,0,2,4.0,NaN,1,2013
4,845,20,86,2014-02-25 14:15:32,NaN,NaN,0,0,1,4.5,NaN,1,2013


In [37]:
query = ("SELECT t1.*,t2.sent FROM "
         "(SELECT no_recipient AS us_user, COUNT(*) AS received FROM `notifications` WHERE no_type = 'feedback request' GROUP BY no_recipient) AS t1 "
         "NATURAL LEFT JOIN (SELECT no_sender AS us_user,COUNT(*) AS sent FROM notifications WHERE no_type = 'feedback response' GROUP BY no_sender) AS t2 "
         "NATURAL JOIN users_userTypes "
         "WHERE ut_user_type = 'formatore'"
        )
export("",cnx,filename="supervisors_feedbacks",abbr=DBABBR,query=query)

246 rows written successfully to ../data/mysql_extractions/supervisors_feedbacks.csv


In [38]:
df = pd.read_csv(f"{data_folder}supervisors_feedbacks.csv")
df.head()

,us_user,received,sent
0,4,1,2.0
1,5,46,62.0
2,6,1,NaN
3,86,70,NaN
4,87,11,19.0


## Logins

In [39]:
export("logins_users_school_year",cnx,filename="logins",abbr=DBABBR)

91058 rows written successfully to ../data/mysql_extractions/logins.csv


and close the connection

In [12]:
cnx.close()

#### EXPORT INI INFO

In [ ]:
#Write changes back to file
with open(inifile, 'w') as conf:
    config_object.write(conf)